# A Guided Tour of Ray Core: Parallel Iterators

[*Parallel Iterators*](https://docs.ray.io/en/latest/iter.html) provide a simple yet powerful API for data ingest and stream processing, where transformations are based on method chaining.

Parallel iterators get partitioned into *data shards*, and Ray creates a worker (an *actor*) to produces the data for each shard.
Evaluation is *lazy*, i.e., only executed when the application calls `next()` to fetch the next item in a sequence.

Parallel iterators are fully serializable, so they can be passed to remote tasks and actors.
In effect, these can be used to operate over infinite sequences of items, with the processing distributed across a cluster.

---

First, let's start Ray…

In [1]:
from icecream import ic
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

File descriptor limit 256 is too low for production servers and may result in connection errors. At least 8192 is recommended. --- Fix with 'ulimit -n 8192'


{'node_ip_address': '192.168.1.65',
 'raylet_ip_address': '192.168.1.65',
 'redis_address': '192.168.1.65:6379',
 'object_store_address': '/tmp/ray/session_2021-02-25_10-41-09_151464_10012/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-25_10-41-09_151464_10012/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-02-25_10-41-09_151464_10012',
 'metrics_export_port': 65309,
 'node_id': '58a492330004964c4c23164712dd62f5c3d785fa'}

## Parallel Iterators

We'll create a parallel iterator from the sequence `items`, using 2 worker actors:

In [2]:
items = [1, 2, 3, 4, 5]

iter1 = ray.util.iter.from_items(items, num_shards=2)
iter1

ParallelIterator[from_items[int, 5, shards=2]]

This `iter1` object can now be passed (i.e., serialized) to remote tasks and remote methods.

To read elements from a parallel iterator, it can be converted to a [`LocalIterator`](https://docs.ray.io/en/latest/iter.html#ray.util.iter.LocalIterator) using two approaches.

Calling [`gather_sync()`](https://docs.ray.io/en/latest/iter.html#ray.util.iter.ParallelIterator.gather_sync)
returns a local iterable for *synchronous* iteration.
In other words, next items will be fetched from the shards on-demand as the application steps through the iterator sequence.

In [3]:
local_iter1 = iter1.gather_sync()
local_iter1

LocalIterator[ParallelIterator[from_items[int, 5, shards=2]].gather_sync()]

In [4]:
for item in local_iter1:
    ic(item)

ic| item: 1
ic| item: 2
ic| item: 3
ic| item: 4
ic| item: 5


When applying a function to the sequence (i.e., some kind of transformation) a parallel iterator provides semantic guarantees for *fetch ordering*. In other words, the transformation is guaranteed to get applied to each element of the sequence before the next item is fetched from the source actor.
For example, this can be useful if you need to update the source actor between iterator steps.

To illustrate a simple case of how to apply a function, first we'll define a class to perform some calculation:

In [5]:
class CumulativeSum:
    def __init__ (self):
        self.total = 0

    def __call__ (self, x):
        self.total += x
        return (self.total, x)

Now apply that class to the sequence of items, using the [`for_each()`](https://docs.ray.io/en/latest/iter.html#ray.util.iter.ParallelIterator.for_each) method:

In [6]:
for x in iter1.for_each(CumulativeSum()).gather_sync():
    print(x)

(1, 1)
(2, 2)
(4, 3)
(6, 4)
(9, 5)


Alternatively, calling [`gather_async()`](https://docs.ray.io/en/latest/iter.html#ray.util.iter.ParallelIterator.gather_async)
returns a local iterable for *asynchronous* iteration.
In other words, next items will be fetched from the shards asynchronously as soon as the previous item gets computed.
In this case, the fetch ordering only applies per shard.

Another way to access a parallel iterator is as a collection of its shards:

In [7]:
iter1.shards()

[LocalIterator[from_items[int, 5, shards=2].shard[0]],
 LocalIterator[from_items[int, 5, shards=2].shard[1]]]

In [8]:
for shard in iter1.shards():
    ic(shard)
    
    for item in shard:
        ic(item)

ic| shard: LocalIterator[from_items[int, 5, shards=2].shard[0]]
ic| item: 1
ic| item: 3
ic| item: 5
ic| shard: LocalIterator[from_items[int, 5, shards=2].shard[1]]
ic| item: 2
ic| item: 4


Note that each shard should only be read by one process at a time.

As an example, let's iterate through the JSON source for the Jupyter notebooks in this repo as if this were a streaming input source.

We'll filter to get the text in markdown cells, evaluated in batches – which creates a sliding window across the input stream:

In [9]:
from pathlib import Path
import numpy as np
import json

nb_items = list(Path(".").glob("ex_*.ipynb"))
window_width = 20

iter2 = (
    ray.util.iter.from_items(nb_items, num_shards=3)
        .for_each(lambda f: json.load(open(f)))
        .for_each(lambda nb: nb["cells"])
        .flatten()
        .for_each(lambda cell: cell["source"] if cell["cell_type"] == "markdown" else [])
        .flatten()
        .for_each(lambda line: 1 if "Ray" in line else 0)
        .batch(window_width)
        .for_each(np.mean)
)

iter2

ParallelIterator[from_items[PosixPath, 6, shards=3].for_each().for_each().flatten().for_each().flatten().for_each().batch(20).for_each()]

Now calculate the probability of the term `Ray` occurring within the lines in each batch:

In [10]:
for freq in iter2.gather_async():
    ic(freq)

ic| freq: 0.2
ic| freq: 0.2
ic| freq: 0.2
ic| freq: 0.2
ic| freq: 0.05
ic| freq: 0.09090909090909091
ic| freq: 0.0
ic| freq: 0.1
ic| freq: 0.0


Let's rework this to show an example of passing iterator shards to remote functions.
We'll define a remote function `nb_word_count` to tally *word count* among the markdown cells in each notebook:

In [11]:
from collections import defaultdict

@ray.remote
def nb_word_count (shard):
    wc = defaultdict(int)
    punct = """'`<>[](){}*.,:…-'"""
    
    for nb_path in shard:
        with open(nb_path) as f:
            nb = json.load(f)
            for cell in nb["cells"]:
                if cell["cell_type"] == "markdown":
                    for line in cell["source"]:
                        for token in line.strip("# ").lower().split():
                            token = token.strip(punct)
                            wc[token] += 1

    return wc

Now pass each of the shards to a remote function:

In [12]:
nb_items = list(Path(".").glob("ex_*.ipynb"))

iter3 = ray.util.iter.from_items(nb_items, num_shards=3)

work = [nb_word_count.remote(shard) for shard in iter3.shards()]

To show the end results, we'll aggregate the word counts calculated from each shard:

In [13]:
wc_sum = defaultdict(int)

for wc in ray.get(work):
    for token, count in wc.items():
        wc_sum[token] += count

Then list the tokens ranked in descending order:

In [14]:
for token, count in sorted(wc_sum.items(), key=lambda item: item[1], reverse=True):
    if count > 1:
        ic(token, count)

ic| token: 'a', count: 55
ic| token: 'the', count: 54
ic| token: 'to', count: 36
ic| token: 'of', count: 23
ic| token: 'ray', count: 21
ic| token: 'remote', count: 21
ic| token: 'this', count: 20
ic| token: 'for', count: 19
ic| token: 'in', count: 19
ic| token: 'and', count: 15
ic| token: 'parallel', count: 13
ic| token: '', count: 12
ic| token: 'example', count: 12
ic| token: "let's", count: 11
ic| token: 'function', count: 11
ic| token: 'be', count: 11
ic| token: "we'll", count: 10
ic| token: 'first', count: 9
ic| token: 'is', count: 9
ic| token: 'on', count: 9
ic| token: 'start', count: 8
ic| token: 'shards', count: 8
ic| token: 'an', count: 8
ic| token: 'can', count: 8
ic| token: 'from', count: 8
ic| token: 'using', count: 8
ic| token: 'use', count: 7
ic| token: 'by', count: 7
ic| token: 'data', count: 7
ic| token: 'each', count: 7
ic| token: 'items', count: 7
ic| token: 'iterator', count: 7
ic| token: 'object', count: 7
ic| token: 'now', count: 7
ic| token: 'as', count: 7
ic| toke

---

## Summary

Parallel iterators provide a somewhat higher-level abstraction which uses Ray actors and `ray.wait` loops, and fit conveninently into efficient software patterns in Python.

Engineering trade-offs are available a multiple levels:

  * trade-off compute and memory requirements by partioning sequences of items into data shards
  * trade-off compute and memory requirements for transformations on items by passing the data shards to remote functions (stateless) and remote methods (stateful)
  * trade-off the semantic guarantees on fetch ordering by using asynchronous iteration